In [ ]:
"""import kagglehub
kagglehub.login()"""

Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
"""isic_2024_challenge_path = kagglehub.competition_download('isic-2024-challenge')"""

100%|██████████| 2.00G/2.00G [01:38<00:00, 21.9MB/s]

Extracting files...


In [ ]:
##############
# LIBRARIES
##############

import pandas as pd
import numpy as np

from sklearn.preprocessing import RobustScaler

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Adjusting Row Column Settings

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)

In [ ]:
############
# LOAD DATA
############

train_data_path = "/content/drive/MyDrive/Skin Cancer Detection/exploratory_data_analysis.csv"
train_data = pd.read_csv(train_data_path)

train_image_dir = f"{isic_2024_challenge_path}/train-image"

In [ ]:
#################
# MISSING VALUES
#################

train_data = train_data[~((train_data['target'] == 0) & (train_data.isnull().any(axis=1)))]

train_data['age_approx'] = train_data['age_approx'].fillna(train_data['age_approx'].median())
train_data['sex'] = train_data['sex'].fillna(train_data['sex'].mode()[0])

<ipython-input-6-9e8c60f408b8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['age_approx'] = train_data['age_approx'].fillna(train_data['age_approx'].median())
<ipython-input-6-9e8c60f408b8>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['sex'] = train_data['sex'].fillna(train_data['sex'].mode()[0])


In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 381926 entries, 0 to 401058
Data columns (total 32 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   target                       381926 non-null  int64  
 1   age_approx                   381926 non-null  float64
 2   sex                          381926 non-null  object 
 3   anatom_site_general          381926 non-null  object 
 4   clin_size_long_diam_mm       381926 non-null  float64
 5   tbp_tile_type                381926 non-null  object 
 6   tbp_lv_A                     381926 non-null  float64
 7   tbp_lv_Aext                  381926 non-null  float64
 8   tbp_lv_B                     381926 non-null  float64
 9   tbp_lv_C                     381926 non-null  float64
 10  tbp_lv_H                     381926 non-null  float64
 11  tbp_lv_Hext                  381926 non-null  float64
 12  tbp_lv_L                     381926 non-null  float64
 13  tbp_

In [ ]:
#####################
# ENCODING / SCALING
#####################

one_hot_cols = ['tbp_tile_type', 'anatom_site_general', 'sex']
train_data = pd.get_dummies(train_data, columns=one_hot_cols, drop_first=True, dtype=int)


columns_to_scale = train_data.select_dtypes(include=['float64', 'int64']).columns.difference(['target', 'images_path'])
scaler = RobustScaler()
train_data[columns_to_scale] = scaler.fit_transform(train_data[columns_to_scale])


# Rename
train_data.rename(columns={'tbp_tile_type_3D: white': 'tbp_tile_type_3D_white'}, inplace=True)

## Preprocessing

In [ ]:
##############
# EXPORT DATA
##############

file_path = '/content/drive/MyDrive/Skin Cancer Detection/preprocessing_data.csv'
train_data.to_csv(file_path, index=False)